In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, os


In [3]:
df_files = os.listdir('../../data/DataGithub/')
df_files

['AntoineAugusti_Vacances_19902026.csv',
 'AntoineAugusti_Vacances_Table_19902026.csv',
 'code_etablissement.csv',
 'Effectifs_ecoles.csv',
 'fetes_chretiennes.csv',
 'fetes_juives.csv',
 'fetes_musulmanes.csv',
 'frequentation_cantines_v3.csv',
 'incidence RDD 3.csv',
 'incidence RDD 6.csv',
 'incidence RDD 7.csv',
 'Journees_de_greve.csv',
 'jours-feries-seuls.csv',
 'key_jours.xlsx',
 'Liste ETS_geo_AE12102020.xlsx',
 'menus_2011-2015.csv',
 'menus_2016-2019.csv',
 'missing_strikes.xlsx',
 'mouvements-sociaux-depuis-2002.csv',
 'mouvements-sociaux-depuis-2002.json',
 'orga_des_etablissements.xlsx',
 'ramadan.csv',
 'vacances_Nantes_2011-2019.csv']

In [4]:
df = pd.read_csv('244400404_effectifs-eleves-cleaned')
df['datetime']= df['Date'].apply( lambda string1 : datetime.datetime.strptime(string1,"%Y-%m-%d"))
df['joursem']= df['datetime'].apply( lambda date1 : date1.weekday())
df['isoweek']= df['datetime'].apply( lambda date1 : date1.isocalendar().week)
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870,2011-09-06,1,36
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698,2011-09-05,0,36
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427,2011-09-05,0,36
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169,2011-09-05,0,36
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957,2011-09-05,0,36


In [5]:
df_calend=pd.read_csv('../../data/SiteNantes/CSV/244400404_nombre-convives-jour-cantine-nantes-2011.csv', sep=';')
df_calend['datetime']= df_calend['Date'].apply( lambda string1 : datetime.datetime.strptime(string1,"%Y-%m-%d")) 
df_calend=df_calend['datetime'].unique()
df_calend = np.sort(df_calend)
df_calend.reshape(-1,1)
df_calend

array(['2011-01-03T00:00:00.000000000', '2011-01-04T00:00:00.000000000',
       '2011-01-06T00:00:00.000000000', ...,
       '2022-05-20T00:00:00.000000000', '2022-05-23T00:00:00.000000000',
       '2022-05-24T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
df_calend3 = pd.DataFrame(df_calend)
df_calend3.rename(columns={0: 'datetime'}, inplace=True)
df_calend3['joursem']= df_calend3['datetime'].apply( lambda date1 : date1.weekday())
df_calend3['isoweek']= df_calend3['datetime'].apply( lambda date1 : date1.isocalendar().week)
df_calend3['an']= df_calend3['datetime'].apply( lambda date1 : date1.year)
# df_calend3.head()

In [7]:
df_ponts=[[df_calend[i],int(min(abs(df_calend[i]-df_calend[i-1]),abs(df_calend[i+1]-df_calend[i]))/np.timedelta64(1, "D")-1)] for i in range(1,len(df_calend)-1) ]
df_ponts=pd.DataFrame(df_ponts)
df_ponts.rename(columns={0: 'datetime', 1: 'datelag'}, inplace=True)

In [8]:
df_lagbefore=[[df_calend[i],int(abs(df_calend[i]-df_calend[i-1])/np.timedelta64(1, "D"))-1] for i in range(1,len(df_calend)) ]
df_lagbefore=pd.DataFrame(df_lagbefore)
df_lagbefore.rename(columns={0: 'datetime', 1: 'lagbefore'}, inplace=True)

In [9]:
df_lagafter=[[df_calend[i],int(abs(df_calend[i+1]-df_calend[i])/np.timedelta64(1, "D"))-1] for i in range(0,len(df_calend)-1) ]
df_lagafter=pd.DataFrame(df_lagafter)
df_lagafter.rename(columns={0: 'datetime', 1: 'lagafter'}, inplace=True)

In [10]:
df_calend3=pd.merge(df_calend3, df_ponts, how='left', on='datetime')
df_calend3=pd.merge(df_calend3, df_lagbefore, how='left', on='datetime')
df_calend3=pd.merge(df_calend3, df_lagafter, how='left', on='datetime')

df_calend3['datelag'].fillna(0, inplace=True)
df_calend3['lagbefore'].fillna(0, inplace=True)
df_calend3['lagafter'].fillna(0, inplace=True)

In [11]:
df_calend3.head()

,datetime,joursem,isoweek,an,datelag,lagbefore,lagafter
0,2011-01-03,0,1,2011,0.0,0.0,0.0
1,2011-01-04,1,1,2011,0.0,0.0,1.0
2,2011-01-06,3,1,2011,0.0,1.0,0.0
3,2011-01-07,4,1,2011,0.0,0.0,2.0
4,2011-01-10,0,2,2011,0.0,2.0,0.0


Let's look whether the data is consistent with 5 vacation times every year

In [12]:
df_calend3[['an','lagbefore']][df_calend3['lagbefore']>8].groupby('an').count()

,lagbefore
an,
2011,4
2012,5
2013,5
2014,5
2015,7
2016,5
2017,5
2018,5
2019,5


Clearly we have an issue in 2015 with 7 vacation times

In [13]:
df_calend3[(df_calend3['an']==2015) & (df_calend3['lagbefore']>8)]

,datetime,joursem,isoweek,an,datelag,lagbefore,lagafter
566,2015-01-05,0,2,2015,0.0,16.0,0.0
586,2015-02-23,0,9,2015,0.0,16.0,0.0
590,2015-04-02,3,14,2015,0.0,33.0,0.0
594,2015-04-27,0,18,2015,0.0,16.0,0.0
613,2015-06-15,0,25,2015,0.0,9.0,0.0
621,2015-09-01,1,36,2015,1.0,62.0,1.0
647,2015-11-02,0,45,2015,0.0,16.0,0.0


We have two Easter vacations and an extra vacation time in June which has nothing to do with the official calendar.
Even as we can rely most of the time on the cantine calendar to find out the vacation times, it doesn't work for 2015. As we don't want to loose 2015, we'll have to look for the offical vacation data.